In [1]:
!unzip "/content/drive-download-20200305T020031Z-001.zip"

unzip:  cannot find or open /content/drive-download-20200305T020031Z-001.zip, /content/drive-download-20200305T020031Z-001.zip.zip or /content/drive-download-20200305T020031Z-001.zip.ZIP.


In [0]:
from gensim.models import Word2Vec
import os
import pandas as pd

In [0]:
path = '/content/data'
def readdata(path):
    list_file = os.listdir(path)
    data = pd.DataFrame()
    for filename in list_file:
        data = pd.concat([data, pd.read_csv(os.path.join(path, filename), sep = ',')])
        
    return data.Review, data.Label

In [4]:
reviews, labels = readdata(path)
input_gensim = []
for review in reviews:
    input_gensim.append(review.split())
    
model = Word2Vec(input_gensim, size=128, window=5, min_count=0, workers=4, sg=1)
model.wv.save("word.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
import gensim.models.keyedvectors as word2vec
model_embedding = word2vec.KeyedVectors.load('./word.model')

word_labels = []
max_seq = 200
embedding_size = 128

for word in model_embedding.vocab.keys():
    word_labels.append(word)
    
def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    lencmt = len(words)

    for i in range(max_seq):
        indexword = i % lencmt
        if (max_seq - i < lencmt):
            break
        if(words[indexword] in word_labels):
            matrix[i] = model_embedding[words[indexword]]
    matrix = np.array(matrix)
    return matrix

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.preprocessing import sequence

sequence_length = 200
embedding_size = 128
num_classes = 3
filter_sizes = 3
num_filters = 150
epochs = 50
batch_size = 30
learning_rate = 0.01
dropout_rate = 0.5

Using TensorFlow backend.


In [7]:
from tqdm import tqdm
train_data = []
label_data = []

for x in tqdm(reviews):
    train_data.append(comment_embedding(x))
train_data = np.array(train_data)

for y in tqdm(labels):
    label_ = np.zeros(3)
    try:
        label_[int(y)] = 1
    except:
        label_[0] = 1
    label_data.append(label_)

100%|██████████| 9727/9727 [00:00<00:00, 446910.31it/s]


In [8]:

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

x_train = train_data.reshape(train_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_train = np.array(label_data)

# Define model

# model = Sequential()
# model.add(Embedding(20000,100,input_shape=(sequence_length, embedding_size, 1), activation='relu'))

# model.add(Dropout(0.2))
# model.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1,activation = 'sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model = keras.Sequential()
model.add(layers.Convolution2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(198, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
# Train model
adam = tf.train.AdamOptimizer()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 1, 150)       57750     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 150)         0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 150)         0         
_________________________________________________________________
flatten (Flatten)            (None, 150)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               19328     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 77,465
Trainable params: 77

In [9]:
model.fit(x = x_train[:7000], y = y_train[:7000], batch_size = batch_size, verbose=1, epochs=epochs, validation_data=(x_train[:3000], y_train[:3000]))

model.save('models.h5')

Train on 7000 samples, validate on 3000 samples
Epoch 1/50
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.8737 - acc: 0.6091 - val_loss: 0.5522 - val_acc: 0.8250
Epoch 2/50
7000/7000 [==============================] - 2s 237us/sample - loss: 0.7480 - acc: 0.6839 - val_loss: 0.4783 - val_acc: 0.8623
Epoch 3/50
7000/7000 [==============================] - 2s 235us/sample - loss: 0.7062 - acc: 0.7026 - val_loss: 0.4524 - val_acc: 0.8907
Epoch 4/50
7000/7000 [==============================] - 2s 238us/sample - loss: 0.6755 - acc: 0.7143 - val_loss: 0.4298 - val_acc: 0.8830
Epoch 5/50
7000/7000 [==============================] - 2s 236us/sample - loss: 0.6599 - acc: 0.7250 - val_loss: 0.4502 - val_acc: 0.8963
Epoch 6/50
7000/7000 [==============================] - 2s 233us/sample - loss: 0.6376 - acc: 0.7347 - val_loss: 0.4048 - val_acc: 0.9013
Epoch 7/50
7000/7000 [==============================] - 2s 233us/sample - loss: 0.6102 - acc: 0.7446 - val_loss: 0.3610 - val_

In [0]:
#from keras.models import load_model
#model_sentiment = load_model("models.h5")

In [10]:
def pre_process(text):
# các bạn tự thêm các tiền xử lý của mình vào đây nhá
    return text

text =input("Enter a comment: ")
text = pre_process(text)

maxtrix_embedding = np.expand_dims(comment_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
if (result==1 and result==0):
  print ('positive')
  value='positive'
else:
    print ('negative')
    value='negative'



data = {'Comment':[text],'Satisfaction':[value]}
df = pd.DataFrame(data)
df.to_csv('/content/comment.csv',index=False, mode ='a', header=False)
df


Enter a comment: khùng
negative


,Comment,Satisfaction
0,khùng,negative


In [0]:
e=pd.read_csv("/content/data/Datasets.csv")
e